In [1]:
import json
# Load JSON file
with open('./data/appl.json', 'r') as file:
    data = json.load(file)

In [2]:
from chroma import build_chroma_from_json
from langchain.docstore.document import Document
from langchain_openai import OpenAIEmbeddings
from langchain_mistralai import MistralAIEmbeddings
from langchain_chroma import Chroma

from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

True

In [3]:
documents = []
for key, value in data.items():
    # Convert the value to a string if it's not already
    content = str(value)
    # Create a Document object
    doc = Document(page_content=content)
    documents.append(doc)

In [4]:
embedding_func = OpenAIEmbeddings(model="text-embedding-3-small")

In [5]:
Chroma.from_documents(
            documents[:7],
            embedding_func,
            persist_directory="./chromas/appl"
            )

In [8]:
import os
from langchain_chroma import Chroma
from langchain.tools.retriever import create_retriever_tool
from langchain_openai import OpenAIEmbeddings

from dotenv import load_dotenv

load_dotenv()

embedding_function = OpenAIEmbeddings(model="text-embedding-3-small")
db = Chroma(
    persist_directory="./chromas/appl",
    embedding_function=embedding_function
    )

retriever = db.as_retriever()

appl_tool = create_retriever_tool(
    retriever,
    "appl_reporting_searcher",
    """Search information about Apple's reporting to the SEC"""
)

In [9]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(model="open-mistral-nemo")

In [16]:
tools = [appl_tool]

In [17]:
import os
import json

from typing import Annotated
from typing_extensions import TypedDict
from dotenv import load_dotenv

from langchain_mistralai import ChatMistralAI
from langchain_mistralai import MistralAIEmbeddings

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

from langchain_core.messages import SystemMessage, AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate



load_dotenv()
mistral_api_key = os.getenv("MISTRAL_API_KEY")


llm = ChatMistralAI(
    model="open-mistral-nemo",
    api_key=mistral_api_key,
    temperature=0.5
    )

llm_with_tools = llm.bind_tools(tools)

class State(TypedDict):
    messages: Annotated[list, add_messages]

def agent(state: State):
    system_message = SystemMessage(content="""
    """)
    response = llm_with_tools.invoke([system_message] + state["messages"])
    return {"messages": state["messages"] + [("assistant", response.content)]}

# Graph setup
graph_builder = StateGraph(State)

# Add nodes
graph_builder.add_node("agent", agent)
graph_builder.add_node("tools", ToolNode(tools=tools))

# Add edges
graph_builder.add_edge(START, "agent")
graph_builder.add_conditional_edges(
    "agent",
    tools_condition
)
graph_builder.add_edge("tools", "agent")

# Compile the graph
graph = graph_builder.compile()

In [ ]:
    conversation = []
    while True:
        user_input = input("You: ")
        if user_input.lower() in ['exit', 'quit', 'bye']:
            print("Goodbye!")
            break
        
        conversation.append(("human", user_input))
        response = graph.invoke({"messages": conversation})
        assistant_message = response["messages"][-1]
        conversation.append(("assistant", assistant_message.content))
        print(f"Assistant: {assistant_message.content}")

In [23]:
graph.invoke({"messages": [("human", "Tell me about apples reports")]})

{'messages': [HumanMessage(content='Tell me about apples reports', id='dedc89d4-51e1-4b7b-85cc-6978286f0a42'),
  AIMessage(content='', id='a233e359-8956-43f7-8bc0-153656421780')]}